In [1]:
import pandas as pd

ratings = pd.read_csv('data/ratings_movies.csv', sep=',')


In [4]:
ratings.tail(5)

,Unnamed: 0,userId,movieId,rating,date,title,genres
100831,100831,610,166534,4.0,2017-05-03 21:53:22,Split (2017),Drama|Horror|Thriller
100832,100832,610,168248,5.0,2017-05-03 22:21:31,John Wick: Chapter Two (2017),Action|Crime|Thriller
100833,100833,610,168250,5.0,2017-05-08 19:50:47,Get Out (2017),Horror
100834,100834,610,168252,5.0,2017-05-03 21:19:12,Logan (2017),Action|Sci-Fi
100835,100835,610,170875,3.0,2017-05-03 21:20:15,The Fate of the Furious (2017),Action|Crime|Drama|Thriller


In [11]:
ratings['date'] = pd.to_datetime(ratings['date'], dayfirst=True)
ratings['date'].tail(5)

100831   2017-05-03 21:53:22
100832   2017-05-03 22:21:31
100833   2017-05-08 19:50:47
100834   2017-05-03 21:19:12
100835   2017-05-03 21:20:15
Name: date, dtype: datetime64[ns]

In [12]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   Unnamed: 0  100836 non-null  int64         
 1   userId      100836 non-null  int64         
 2   movieId     100836 non-null  int64         
 3   rating      100836 non-null  float64       
 4   date        100836 non-null  datetime64[ns]
 5   title       100836 non-null  object        
 6   genres      100836 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(2)
memory usage: 5.4+ MB


In [13]:
#библиотека для регулярных выражений
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

In [25]:
ratings['year_release'] = ratings['title'].apply(lambda x: get_year_release(x))

#ratings['year_release'] = ratings['year_release'].astype(int)

ratings['year_release'].tail(5)



100831    2017.0
100832    2017.0
100833    2017.0
100834    2017.0
100835    2017.0
Name: year_release, dtype: float64

In [26]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Unnamed: 0    100836 non-null  int64         
 1   userId        100836 non-null  int64         
 2   movieId       100836 non-null  int64         
 3   rating        100836 non-null  float64       
 4   date          100836 non-null  datetime64[ns]
 5   title         100836 non-null  object        
 6   genres        100836 non-null  object        
 7   year_release  100818 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 6.2+ MB


In [27]:
100836-100818

18

In [37]:
# Какой фильм, выпущенный в 1999 году, получил наименьшую среднюю оценку зрителей?

mask = ratings['year_release'] == 1999

ratings[mask].groupby(by='title').mean()['rating'].sort_values().index[0]

C:\Users\trigubov\AppData\Local\Temp\ipykernel_1968\2824925257.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ratings[mask].groupby(by='title').mean()['rating'].sort_values().index[0]


'Bloodsport: The Dark Kumite (1999)'

In [39]:
mask = ratings['year_release'] == 2010

ratings[mask].groupby(by='genres').mean()['rating'].sort_values().index[0]

C:\Users\trigubov\AppData\Local\Temp\ipykernel_1968\3513594969.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ratings[mask].groupby(by='genres').mean()['rating'].sort_values().index[0]


'Action|Sci-Fi'

In [47]:
# Какой пользователь (userId) посмотрел наибольшее количество различных (уникальных) комбинаций жанров (genres) фильмов?
ratings.groupby(by='userId')['genres'].nunique().sort_values(ascending=False).index[0]

599

In [54]:
# Найдите пользователя, который выставил наименьшее количество оценок, но его средняя оценка фильмам наибольшая.
# В качестве ответа укажите идентификатор этого пользователя.
# Чтобы рассчитать несколько параметров для каждого пользователя (количество оценок и среднюю оценку), можно воспользоваться методом agg() на сгруппированных данных.

ratings.groupby(by='userId')['rating'].agg(['count','mean']).sort_values(
    by=['count', 'mean'],
    ascending=[True, False]
    )


,count,mean
userId,,
53,20,5.000000
595,20,4.200000
189,20,4.100000
569,20,4.000000
278,20,3.875000
...,...,...
274,1346,3.235884
448,1864,2.847371
474,2108,3.398956


In [62]:
mask1 = ratings['year_release'] == 2018
rat1 = ratings[mask1].groupby(by='genres')['rating'].agg(['count','mean'])

mask2 = rat1['count'] > 10

rat1[mask2]['mean'].sort_values(ascending=False).index[0]

'Action|Adventure|Sci-Fi'

In [110]:
ratings['year_rating'] = ratings['date'].dt.year
#ratings.groupby(by='year_rating')['rating'].mean()

pivot = ratings.pivot_table(
    values='rating',
    aggfunc='mean',
    columns ='genres',
    index='year_rating',    
    fill_value=0
)

#display (pivot)

# pivot['Action|Adventure'].min()

# pivot['Action|Adventure|Animation|Children|Comedy|IMAX'].sort_values(ascending=False).index[0]

#x = pivot.iloc[22].sort_values(ascending=False)
#mask = x == 5
#'Animation|Children|Mystery' in x[mask]

#pivot['Comedy']

pivot.loc[2018]

genres
(no genres listed)                     3.676471
Action                                 2.588235
Action|Adventure                       3.397436
Action|Adventure|Animation             3.650000
Action|Adventure|Animation|Children    3.166667
                                         ...   
Sci-Fi|Thriller                        3.562500
Sci-Fi|Thriller|IMAX                   0.000000
Thriller                               3.296296
War                                    0.000000
Western                                3.611111
Name: 2018, Length: 951, dtype: float64

In [118]:
orders = pd.read_csv('data/orders.csv', sep=';')
products = pd.read_csv('data/products.csv', sep=';')
display(orders)
display(products.head(5))

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5
5,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,103,5
6,08.11.2019 08:36:22,5,5,Отменён,Нет,Да,Нет,124,1
7,08.11.2019 08:36:22,4,9,"Принят, ожидается оплата",Нет,Нет,Да,91,1
8,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,103,3
9,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,104,3


,Product_ID,Name,Price,CURRENCY
0,47,Шатны Полосатый рейс,2999,RUR
1,51,Платье Аленький цветочек,4999,RUR
2,53,Штаны Цветочная Поляна,4999,RUR
3,71,Платье Ночная Жизнь,7999,RUR
4,74,Платье Ночная Жизнь XXXL,8999,RUR


In [117]:
orders_products = orders.merge(
    products,
    left_on='ID товара',
    right_on='Product_ID',
    how='left'
)
orders_products

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Product_ID,Name,Price,CURRENCY
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,103.0,"Носки Подарочные, муж",199.0,RUR
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100,86.0,"Носки Простые, муж",45.0,RUR
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10,104.0,"Носки Подарочные, жен",249.0,RUR
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7,104.0,"Носки Подарочные, жен",249.0,RUR
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5,104.0,"Носки Подарочные, жен",249.0,RUR
5,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,103.0,"Носки Подарочные, муж",199.0,RUR
6,08.11.2019 08:36:22,5,5,Отменён,Нет,Да,Нет,124,1,124.0,Носки беговые Camino,999.0,RUR
7,08.11.2019 08:36:22,4,9,"Принят, ожидается оплата",Нет,Нет,Да,91,1,91.0,"Носки Честные, муж",50.0,RUR
8,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,103,3,103.0,"Носки Подарочные, муж",199.0,RUR
9,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,104,3,104.0,"Носки Подарочные, жен",249.0,RUR


In [128]:
orders_products['Profit']=(orders_products['Price']*orders_products['Количество'])
mask = orders_products['Оплачен'] == 'Да'
orders_products[mask].groupby(by='ID Покупателя')['Profit'].sum().sort_values(ascending=False)

ID Покупателя
7    17096.0
5    13043.0
8     1344.0
1        0.0
Name: Profit, dtype: float64